<a href="https://colab.research.google.com/github/iannellif/churn/blob/main/eth_handson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

# Load Data

In [ ]:
import requests
import tarfile
import io
import pandas as pd
import os

url = "https://raw.githubusercontent.com/iannellif/churn/main/eth_df_churn.tar.gz"

# Download the tar.gz file
response = requests.get(url)
tar_file = tarfile.open(fileobj=io.BytesIO(response.content), mode="r:gz")

# Extract the CSV file
tar_file.extractall("/content/churn_data")
tar_file.close()

# List extracted files
extracted_files = os.listdir("/content/churn_data")
print("Extracted files:", extracted_files)

df = pd.read_csv("/content/churn_data/eth_df_churn.csv")

print("\nData loaded successfully. Shape:", df.shape)
print("\nFirst few rows:")
df.head()

In [ ]:
# Define the disengagement threshold for the label
decrease_percent = 0.25

# Function to set negative values to 0
def set_negative_to_zero(value):
    return max(value, 0)

# Apply the function to set negative values to 0 in '0Q' and '1Q'
df['0Q'] = df['0Q'].apply(set_negative_to_zero)
df['1Q'] = df['1Q'].apply(set_negative_to_zero)

# Create the flag column based on the decrease percentage
def calculate_flag(current, previous):
    if previous == 0:
        return 0  # Avoid division by zero
    # percentage change between the current value and the previous value
    change = (current - previous) / previous
    return 1 if change < -decrease_percent else 0

df['0Q_FLAG'] = df.apply(lambda row: calculate_flag(row['0Q'], row['-4Q']), axis=1)
df['1Q_FLAG'] = df.apply(lambda row: calculate_flag(row['1Q'], row['-3Q']), axis=1)


def categorize(row):
    if row['0Q_FLAG'] == 0:
        return 'NEGATIVE'
    elif row['0Q_FLAG'] == 1 and row['1Q_FLAG'] == 1:
        return 'TRUE POSITIVE'
    elif row['0Q_FLAG'] == 1 and row['1Q_FLAG'] == 0:
        return 'FALSE POSITIVE'
    else:
        return 'UNKNOWN'

df['CAT'] = df.apply(categorize, axis=1)

# Remove future flags and spending
del df['1Q']
del df['0Q_FLAG']
del df['1Q_FLAG']

df

In [ ]:
df['CAT'].value_counts()

In [ ]:
df.describe()

In [ ]:
(df.isna().sum()/len(df)*100).plot(kind = 'barh', figsize = (10,10))
plt.title('Percentage of Missing Values')
plt.show()

In [ ]:
df.fillna(0, inplace=True)

(df.isna().sum()/len(df)*100).plot(kind = 'barh', figsize = (10,10))
plt.title('Percentage of Missing Values')
plt.show()

In [ ]:
categorical_cols = df.select_dtypes(include=['object']).columns
# skip the label columns
categorical_cols = categorical_cols.drop(['CAT'])
categorical_cols = categorical_cols.tolist()
categorical_cols

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# One-hot encoding
encoder = OneHotEncoder(sparse=False)
encoded_cols = encoder.fit_transform(df[categorical_cols])
encoded_cols = pd.DataFrame(encoded_cols, columns=encoder.get_feature_names_out(categorical_cols))
encoded_cols.head()

In [ ]:
# Dropping original categorical columns and adding encoded ones
dffdrop = df.drop(categorical_cols, axis=1)
df = pd.concat([dffdrop, encoded_cols], axis=1)
df

# Features and Labels

In [ ]:
df_model = df.copy()

labels = df_model['CAT'].map({'TRUE POSITIVE': 1, 'FALSE POSITIVE': 0, 'NEGATIVE' : 0})

features = df_model.drop(columns=['CAT']).iloc[:, np.r_[0:27]]
# features = df_model.drop(columns=['CAT']).iloc[:, np.r_[0:27,51:len(df_model.columns)-1]]

print(features.columns)

In [ ]:
# Compute the correlation matrix
corr = features.corr()

mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(11, 9))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap='viridis', vmax=1., vmin=-1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .8})

plt.show()

In [ ]:
labels.value_counts()

In [ ]:
tags = 'NEGATIVE', 'POSITIVE'
sizes = [labels[labels==0].count(), labels[labels==1].count()]
explode = (0, 0.1)
fig1, ax1 = plt.subplots(figsize=(6, 4))
ax1.pie(sizes, explode=explode, labels=tags, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("Churn Ratio", size = 20)
plt.show()

In [ ]:
tags = ['NEGATIVE', 'TRUE POSITIVE', 'FALSE POSITIVE']
sizes = [df['CAT'][df['CAT'] == 'NEGATIVE'].count(),
         df['CAT'][df['CAT'] == 'TRUE POSITIVE'].count(),
         df['CAT'][df['CAT'] == 'FALSE POSITIVE'].count()]
explode = (0, 0.1, 0.1)

fig1, ax1 = plt.subplots(figsize=(6, 4))
ax1.pie(sizes, explode=explode, labels=tags, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("Churn Category Distribution", size=20)
plt.show()

# Train data split and scale

In [ ]:
from sklearn.model_selection import train_test_split

Sample division into train, validation, and test datasets. There is no predefined rule; it largely depends on the total dataset size. Typically, it’s 60:20:20, but with large datasets, it can go up to 98:1:1.

In [ ]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
print('Training data shape: ',features_train.shape)
print('Training targets shape: ',labels_train.shape)
print('Test data shape: ',features_test.shape)
print('Test targets shape: ',labels_test.shape)

# ML Pipeline

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from lightgbm import LGBMClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay

In [ ]:
from sklearn.pipeline import Pipeline

In [ ]:
# Define the pipelines
pipelines = {
    'nb': Pipeline([('s', StandardScaler()),
                    ('nb', GaussianNB())]),
    'dt': Pipeline([('s', StandardScaler()),
                    ('dt', DecisionTreeClassifier())]),
    # 'rf': Pipeline([('s', StandardScaler()),
    #                 ('rf', RandomForestClassifier())]),
    'lb': Pipeline([('s', StandardScaler()),
                    ('lb', LGBMClassifier())])
}


print('# Fit the models')
for model in pipelines.values():
    print(model)
    model.fit(features_train, labels_train)

print('# Predict and evaluate')
for name, model in pipelines.items():
    y_pred = model.predict(features_test)

    print(classification_report(labels_test, y_pred))

    accuracy = accuracy_score(labels_test, y_pred)

    print(f"Model: {name}, Accuracy: {accuracy}")

    print('Recall:', recall_score(labels_test, y_pred, average=None))

    print('Precision:', precision_score(labels_test, y_pred, average=None))

    cm = confusion_matrix(labels_test, y_pred)
    print('Confusion Matrix:\n', cm)
    ConfusionMatrixDisplay(confusion_matrix=cm).plot(values_format='')
    plt.show()
    # cm = confusion_matrix(labels_test, y_pred,normalize = 'pred')
    # ConfusionMatrixDisplay(confusion_matrix=cm).plot(values_format='.0%')
    # plt.show()
    # cm = confusion_matrix(labels_test, y_pred,normalize = 'true')
    # ConfusionMatrixDisplay(confusion_matrix=cm).plot(values_format='.0%')
    # plt.show()

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve

In [ ]:
# Plot combined ROC curves
plt.figure(figsize=(6, 3))
for name, model in pipelines.items():
    # Calculate ROC-AUC and plot ROC curve
    if hasattr(model.named_steps[model.steps[-1][0]], "predict_proba"):
        y_prob = model.predict_proba(features_test)[:, 1]
        fpr, tpr, _ = roc_curve(labels_test, y_prob)
        roc_auc = auc(fpr, tpr)

        plt.plot(fpr, tpr, lw=2, label=f'{name} (area = {roc_auc:.2f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic - All Models')
plt.legend(loc="lower right")
plt.show()

# Plot combined precision-recall curves
plt.figure(figsize=(6, 3))
for name, model in pipelines.items():
    if hasattr(model.named_steps[model.steps[-1][0]], "predict_proba"):
        y_prob = model.predict_proba(features_test)[:, 1]
        precision, recall, _ = precision_recall_curve(labels_test, y_prob)
        pr_auc = auc(recall, precision)

        plt.plot(recall, precision, lw=2, label=f'{name} (area = {pr_auc:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve - All Models')
plt.legend(loc="lower left")
plt.show()

# PyTorch ANN

In [ ]:
# pytorch
import torch
print(torch.__version__)

Using the GPU. If a GPU is available, it can be used to accelerate operations. This requires moving tensors to the GPU when performing calculations. It is advisable to check if the GPU is available and, if so, set an appropriate variable for subsequent use.

In [ ]:
# let's check if GPU is available and which type
if torch.cuda.is_available():
  print('Available GPUs number: ',torch.cuda.device_count())
  for i in range(0,torch.cuda.device_count()):
    print(torch.cuda.get_device_name(0))

# if GPU is avialable let's set device='cuda', else 'cpu
device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Computation device: {device}\n")

## Data Preprocessing and Conversion to PyTorch Tensors

This code cell performs several key preprocessing steps on the dataset:

1. **Standardization**: A `StandardScaler` object is created to standardize the features by removing the mean and scaling to unit variance. The scaler is fitted on the training data and then used to transform both the training and test datasets.
2. **Conversion to PyTorch Tensors**: The standardized features and labels from the pandas dataframes are converted into PyTorch tensors, which are required for training a neural network using PyTorch.

These steps ensure that the data is properly scaled and in the correct format for model training and evaluation.


In [ ]:
# Create a StandardScaler object
scaler = StandardScaler()

# Fit on your training data and transform both train and test data
features_train = scaler.fit_transform(features_train)
features_test = scaler.transform(features_test)

# Convert the pandas dataframes into PyTorch tensors
train_data = torch.tensor(features_train, dtype=torch.float)
train_labels = torch.tensor(labels_train.to_numpy(), dtype=torch.long)

test_data = torch.tensor(features_test, dtype=torch.float)
test_labels = torch.tensor(labels_test.to_numpy(), dtype=torch.long)

NOTE: Normalization of input features: Normalizing the input allows for the use of larger learning rates (faster training) and stabilizes the training process.

# Neural network model

## Shallow MLP

Architecture: Shallow MLP with dense layers (Linear in PyTorch): With a single hidden layer using ReLU activations, and 2 output neurons with softmax activation predicting the probability that the customer churned or not:

*   input layer: #features neurons
*   hidden layer: 1024 neurons, ReLU activation
*   output layer: 2 neurons, softmax activation

In [ ]:
inputneurons = features.shape[1]
inputneurons

In [ ]:
# define the 1st layer neurons
layer1neurons = 512

In [ ]:
from torch.utils.data import Dataset

# define the class CustomDataset that inherits from PyTorch’s Dataset class
class CustomDataset(Dataset):
    # constructor method that initializes the dataset object
    def __init__(self, features, labels):
        # converts the features input into a PyTorch tensor of type float
        self.data = torch.tensor(features, dtype=torch.float)
        # converts the labels input into a PyTorch tensor of type long
        self.targets = torch.tensor(labels.to_numpy(), dtype=torch.long)

    def __len__(self):
        # returns the number of samples in the dataset
        return len(self.data)

    def __getitem__(self, idx):
        # retrieves the feature and target at the specified index idx
        return self.data[idx], self.targets[idx]

# Create your datasets:
# train_data and test_data are instances of the CustomDataset class,
# created using training and testing features and labels
train_data = CustomDataset(features_train, labels_train)
test_data = CustomDataset(features_test, labels_test)

This custom dataset can now be used with PyTorch’s data loaders to efficiently load data during model training and evaluation

In [ ]:
print('Training data shape: ',train_data.data.shape)
print('Training targets shape: ',train_data.targets.shape)
print('Test data shape: ',test_data.data.shape)
print('Test targets shape: ',test_data.targets.shape)

In [ ]:
# first customer features and label of the training sample
customer = (train_data.data)[0]
label = (train_data.targets)[0]

print(type(customer)) # tensor torch
print(type(label)) # tensor torch

In [ ]:
print('shape customer: ', customer.shape)
print('shape customer: ', customer.numpy().shape)

In [ ]:
print(customer)
print(label)

In [ ]:
print((train_data[0])[0])

In [ ]:
from torch import nn
import torch.nn.functional as F

# In PyTorch, a neural network is implemented by creating a Python class that inherits
# from PyTorch's nn.Module class and implements two basic methods:
# __init__: definition of the layers used
# forward: function that computes y = ANN(x)

In [ ]:
class ShallowMLP(nn.Module):
    def __init__(self, input_dim=inputneurons, output_dim=2, hidden_dim=layer1neurons):
        super(ShallowMLP, self).__init__()

        self.layer1 = nn.Linear(input_dim, hidden_dim)  #hidden layer
        self.layer2 = nn.Linear(hidden_dim, output_dim) #output layer

    def forward(self, x):
        x = self.layer1(x)
        x = F.relu(x)
        # x = F.tanh(x)
#         x = F.sigmoid(x)
        out = self.layer2(x)
        return out

In [ ]:
# we can do the same for a deep ANN
class DeepMLP(nn.Module):
    def __init__(self, input_dim=inputneurons, output_dim=2, hidden_dim1=layer1neurons, hidden_dim2=layer1neurons):
        super(DeepMLP, self).__init__()

        self.layer1 = nn.Linear(input_dim, hidden_dim1)  # first hidden layer
        self.layer2 = nn.Linear(hidden_dim1, hidden_dim2)  # second hidden layer
        self.layer3 = nn.Linear(hidden_dim2, output_dim)  # output layer

    def forward(self, x):
        x = F.relu(self.layer1(x))  # activation function for first hidden layer
        x = F.relu(self.layer2(x))  # activation function for second hidden layer
        out = self.layer3(x)  # no activation function for the output layer
        return out

In [ ]:
from torchsummary import summary

In [ ]:
# printout the model

model = ShallowMLP()
model = DeepMLP()
print(model)

if torch.cuda.is_available():
  summary(model.cuda(), input_size=(1,inputneurons))
else:
  summary(model, input_size=(1,inputneurons))

## Training and Validation

In [ ]:
# Train-Validation:
# We split the dataset into two subsets to be used in training as
# the training set and validation set, respectively. The validation set is used to tune
# the network's hyperparameters and evaluate the model’s performance on unseen data.

tr_data, val_data = torch.utils.data.random_split(train_data, [train_data.targets.shape[0] - test_data.targets.shape[0], test_data.targets.shape[0]])

# Mini-Batches:
# When training a neural network using SGD (Stochastic Gradient Descent), especially with large datasets,
# it’s computationally efficient to divide the data into smaller chunks called mini-batches.
# This allows the model to update its parameters more frequently and can lead to faster convergence.
# This process is handled automatically in PyTorch through helper functions called Data Loaders.

batch = 100

# Data Loaders:
# In PyTorch, DataLoader is a utility that abstracts the complexity of managing mini-batches,
# reshuffling, and applying transformations. It provides an efficient way to iterate over the dataset.
# Data transformations (like normalization, augmentation, etc.) are often applied to the training data
# to improve the model’s performance and robustness.
# To ensure that the model generalizes well and does not overfit to the order of the training data,
# it’s common practice to reshuffle the data at the beginning of each epoch.
# We use three loader (train, val and test) (or we could use a dictionary to group them and read them at once)

from torch.utils.data import DataLoader

train_dl = DataLoader(tr_data,
                      batch_size=batch,
                      shuffle=True,
                      num_workers=0,
                      drop_last=True)

vali_dl = DataLoader(val_data,
                      batch_size=batch,
                      shuffle=True,
                      num_workers=0,
                      drop_last=True)

test_dl = DataLoader(test_data,
                     batch_size=batch,
                     shuffle=True,
                     num_workers=0)

In [ ]:
# next(iter(train_dl)) retrieves the first batch of data from the train_dl data loader.
# Let's test a batch on the 'untrained' model to check that everything works well

feat, label = next(iter(train_dl))
feat, label

In [ ]:
feat.shape, label.shape

In [ ]:
# Move the feat (features) and label (labels) tensors to the specified device (e.g., CPU or GPU)
# Moving tensors to the appropriate device ensures that computations are performed on the GPU if available,
# which can significantly speed up training and inference.
feat=feat.to(device)
label=label.to(device)

# Pass the features (feat) through the model to get the output (out)
out = model(feat)

# The shape of the prediction is a batch*classes_number tensor
print(out.shape)

# First element of the output tensor, i.e. the predictions for the first sample in the batch.
# The values are the raw output (logits) from the model for the first sample in the batch
print(out[0])

In [ ]:
# The logits are not probabilities but can be converted to probabilities using a softmax function
# This applies the softmax function along the second dimension (the class dimension) of the output tensor, converting the logits to probabilities
probs = F.softmax(out, dim=1)
print(probs[0])

In [ ]:
# To complete the model, we need to define the loss function, any metrics to monitor the
# training of the network, and finally the optimizer

loss_func = nn.CrossEntropyLoss() #cross entropy loss
# loss_func = nn.NLLLoss() #alternatively with log_softmax (same result)


def accuracy(yhat, y):
    #predictions == neurons with the highest probability
    #computes the maximum value along dimension 1 (the class dimension in classification) for each sample in yhat
    preds = torch.max(yhat,1)[1]

    #count the number of True values, giving the total number of correct predictions in the batch
    batch_acc = (preds == y).sum()
    return batch_acc


# !pip install torchmetrics

# from torchmetrics.classification import MulticlassPrecision, MulticlassRecall

# # Initialize metrics with the appropriate task
# precision = MulticlassPrecision(num_classes=2)
# recall = MulticlassRecall(num_classes=2)


def precision(yhat, y):
    preds = torch.max(yhat, 1)[1]
    tp = torch.sum((preds == 1) & (y == 1)).float()
    fp = torch.sum((preds == 1) & (y == 0)).float()

    prec = tp / (tp + fp) if tp + fp > 0 else torch.tensor(0.0).to(y.device)
    return prec

def recall(yhat, y):
    preds = torch.max(yhat, 1)[1]
    tp = torch.sum((preds == 1) & (y == 1)).float()
    fn = torch.sum((preds == 0) & (y == 1)).float()

    rec = tp / (tp + fn) if tp + fn > 0 else torch.tensor(0.0).to(y.device)
    return rec

In [ ]:
# optimizer: for example, we use stochastic gradient descent with momentum,
# that helps accelerate SGD in the relevant direction and dampens oscillations.
# Momentum helps overcome small local minima. Without it, the optimizer might
# get stuck more easily.
# The weights and biases of the neural network that need to be updated
# during training are defined by model.parameters() and the learning rate lr
# which controls how much to change the model parameters in response
# to the estimated error each time the model weights are updated

from torch import optim
opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
# opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))

In [ ]:
# if available we move the model on the GPU
model.to(device)
print(next(model.parameters()).device)

In [ ]:
import time

# In pytorch we need to write the training loop, i.e. the loop on the epochs of training.
# In each epoch we read all events of the dataset and update the nn weigths after each mini-batch
# number of epochs
epochs = 100

# lists to save the value of the loss and the metrics at each epoch to be able to plot them as a function
# of the epoch at the end of training
hist_time = []
hist_loss = []
hist_accuracy = []
hist_precision = []
hist_recall = []
hist_vloss = []
hist_vaccuracy = []
hist_vprecision = []
hist_vrecall = []

# loop over epochs
for epoch in range(epochs):
    print(f"\nEpoch: {epoch + 1}")
    t0 = time.time()
    # training step (where we update the weights of the neural network)
    model.train()
    train_loss = 0
    train_accuracy = 0
    train_precision = 0
    train_recall = 0
    counter = 0
    for xb, yb in train_dl:
        counter += 1
        xb = xb.to(device)  # copy the mini-batch of data to the CPU/GPU
        yb = yb.to(device)  # copy the mini-batch of labels to the CPU/GPU
        pred = model(xb)  # model prediction
        # calculate loss and metrics
        loss = loss_func(pred, yb)
        batch_accuracy = accuracy(pred, yb)
        batch_precision = precision(pred, yb)
        batch_recall = recall(pred, yb)
        # update total loss and metrics
        train_loss += loss.item()
        train_accuracy += batch_accuracy.item()
        train_precision += batch_precision.item()
        train_recall += batch_recall.item()
        # backpropagation
        opt.zero_grad()  # reset gradients before performing backpropagation
        loss.backward()  # calculate the gradients of the loss
        opt.step()  # update the weights
    train_loss /= counter
    train_accuracy /= (counter * batch)
    train_precision /= counter
    train_recall /= counter
    hist_loss.append(train_loss)
    hist_accuracy.append(train_accuracy)
    hist_precision.append(train_precision)
    hist_recall.append(train_recall)

    # validation step (weights are not updated)
    model.eval() # ensures that the model behaves consistently during evaluation, giving the same output for the same input
    vali_loss = 0
    vali_accuracy = 0
    vali_precision = 0
    vali_recall = 0
    counter = 0
    with torch.no_grad():  # prevent weights from being updated
        for xb, yb in vali_dl:
            counter += 1
            xb = xb.to(device)
            yb = yb.to(device)
            pred = model(xb)  # model prediction
            # calculate loss and metrics
            vloss = loss_func(pred, yb)
            batch_accuracy = accuracy(pred, yb)
            batch_precision = precision(pred, yb)
            batch_recall = recall(pred, yb)
            vali_loss += vloss.item()
            vali_accuracy += batch_accuracy.item()
            vali_precision += batch_precision.item()
            vali_recall += batch_recall.item()
    vali_loss /= counter
    vali_accuracy /= (counter * batch)
    vali_precision /= counter
    vali_recall /= counter
    hist_vloss.append(vali_loss)
    hist_vaccuracy.append(vali_accuracy)
    hist_vprecision.append(vali_precision)
    hist_vrecall.append(vali_recall)

    elapsed_time = time.time() - t0
    hist_time.append(elapsed_time)
    print(f"Time: {elapsed_time:.4f}s")
    print(f"Train -> Loss: {train_loss:.6f}, Accuracy: {train_accuracy:.6f}, Precision: {train_precision:.6f}, Recall: {train_recall:.6f}")
    print(f"Val   -> Loss: {vali_loss:.6f}, Accuracy: {vali_accuracy:.6f}, Precision: {vali_precision:.6f}, Recall: {vali_recall:.6f}")

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(range(1,len(hist_loss)+1), hist_loss, color='blue', linestyle='-', label='training', lw=2)
plt.plot(range(1,len(hist_vloss)+1), hist_vloss, color='green', linestyle='-', label='validation', lw=2)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.figure(figsize=(5, 3))
plt.plot(range(1,len(hist_accuracy)+1),hist_accuracy, color='blue', linestyle='-', label='training', lw=2)
plt.plot(range(1,len(hist_vaccuracy)+1),hist_vaccuracy, color='green', linestyle='-', label='validation', lw=2)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.figure(figsize=(5, 3))
plt.plot(range(1,len(hist_precision)+1),hist_precision, color='blue', linestyle='-', label='training', lw=2)
plt.plot(range(1,len(hist_vprecision)+1),hist_vprecision, color='green', linestyle='-', label='validation', lw=2)
plt.xlabel('Epochs')
plt.ylabel('Precision')
plt.legend()
plt.show()

plt.figure(figsize=(5, 3))
plt.plot(range(1,len(hist_recall)+1),hist_recall, color='blue', linestyle='-', label='training', lw=2)
plt.plot(range(1,len(hist_vrecall)+1),hist_vrecall, color='green', linestyle='-', label='validation', lw=2)
plt.xlabel('Epochs')
plt.ylabel('Recall')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(5, 3))
plt.plot(range(1,len(hist_time)+1),hist_time, color='red', linestyle='-', label='training', lw=2)
plt.xlabel('Epochs')
plt.ylabel('CPU Time')
# plt.legend()
plt.show()

# Test the model

In [ ]:
# torch.save(model, 'torch_trained_eth.pt')
# model = torch.load('./torch_trained_eth.pt')
model

In [ ]:
# Some layers like dropout and batch normalization behave differently
# during training and inference.
# eval() ensures they behave correctly for inference
model.eval()

# for the inference it is not necessay to run on GPU
model.to(torch.device('cpu'))

# Initialize variables for metrics
test_loss = 0
test_accuracy = 0
test_precision = 0
test_recall = 0
counter = 0

# Initialize empty lists for true and predicted labels
true_labels = []
predicted_labels = []

with torch.no_grad():  # Disable gradient computation
    for xb, yb in test_dl:
        counter += 1
        xb = xb.to('cpu')
        yb = yb.to('cpu')
        pred = model(xb)

        # Calculate metrics
        test_loss += loss_func(pred, yb).item()
        test_accuracy += accuracy(pred, yb).item()
        test_precision += precision(pred, yb).item()
        test_recall += recall(pred, yb).item()

        # Convert predicted labels to numpy array
        pred_labels = torch.max(pred, 1)[1].numpy()
        true_labels.extend(yb.numpy())
        predicted_labels.extend(pred_labels)

# Calculate average metrics
test_loss /= counter
test_accuracy /= (counter * batch)
test_precision /= counter
test_recall /= counter

# Print results
print(f'Test loss: {test_loss:.6f}')
print(f'Test accuracy: {test_accuracy:.6f}')
print(f'Test precision: {test_precision:.6f}')
print(f'Test recall: {test_recall:.6f}')

# Compute F1 score
f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)
print(f'Test F1 score: {f1_score:.6f}')

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
ConfusionMatrixDisplay(confusion_matrix=conf_matrix).plot(values_format='')
plt.title('Confusion Matrix')
plt.show()